<a href="https://colab.research.google.com/github/TharinsaMudalige/Neuron-Brain_Tumor_Detection_Classification_with_XAI/blob/Detection-Classification-VIT/Brain_Tumour_Classification_Using_VIT_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import tensorflow as tf
from PIL import Image
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Define dataset directory
base_dir = "/content/drive/MyDrive/DSGP_BrainTumorDetection/Preprocessed_Dataset_classes_morepreprocess_techniques"


In [3]:
def load_dataset(base_dir):
    """Load images and labels from the dataset."""
    images = []
    labels = []

    # Ignore hidden folders
    class_names = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d)) and not d.startswith('.')])

    print(f"Class Names: {class_names}")

    for label, class_name in enumerate(class_names):
        class_dir = os.path.join(base_dir, class_name)
        print(f"Processing class: {class_name}, Label: {label}")

        for file in os.listdir(class_dir):
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                file_path = os.path.join(class_dir, file)
                try:
                    image = tf.keras.preprocessing.image.load_img(
                        file_path, color_mode='grayscale', target_size=(224, 224)
                    )
                    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
                    images.append(image)
                    labels.append(label)
                except Exception as e:
                    print(f"[ERROR] Failed to process {file_path}: {e}")

    print(f"Loaded {len(images)} images.")

    images_np = np.array(images, dtype=np.float32)
    labels_np = np.array(labels, dtype=np.int32)

    return images_np, labels_np, class_names


In [ ]:
# Load the dataset
images, labels, class_names = load_dataset(base_dir)
print(f"Dataset loaded: {images.shape}, {labels.shape}")

Class Names: ['Astrocitoma', 'Carcinoma', 'Ependimoma', 'Ganglioglioma', 'Germinoma', 'Glioblastoma', 'Granuloma', 'Meduloblastoma', 'Neurocitoma', 'Oligodendroglioma', 'Papiloma', 'Schwannoma', 'Tuberculoma', 'meningioma', 'no_tumour', 'pituitary']
Processing class: Astrocitoma, Label: 0
Processing class: Carcinoma, Label: 1
Processing class: Ependimoma, Label: 2
Processing class: Ganglioglioma, Label: 3
Processing class: Germinoma, Label: 4
Processing class: Glioblastoma, Label: 5
Processing class: Granuloma, Label: 6
Processing class: Meduloblastoma, Label: 7
Processing class: Neurocitoma, Label: 8
Processing class: Oligodendroglioma, Label: 9
Processing class: Papiloma, Label: 10
Processing class: Schwannoma, Label: 11
Processing class: Tuberculoma, Label: 12
Processing class: meningioma, Label: 13
Processing class: no_tumour, Label: 14
Processing class: pituitary, Label: 15
Loaded 32000 images.


In [ ]:
# Split the dataset equally for each class
train_images, test_images, train_labels, test_labels = train_test_split(
    images, labels, test_size=0.3, random_state=42, stratify=labels
)

for label in range(len(class_names)):  # Iterate over each class label
    class_indices = np.where(labels == label)[0]  # Get indices of all images belonging to the current class
    class_images = images[class_indices]  # Extract images for the current class
    class_labels = labels[class_indices]  # Extract labels for the current class
    # Split the class data into training and testing sets (80% train, 20% test)
    train_x, test_x, train_y, test_y = train_test_split(class_images, class_labels, test_size=0.3, random_state=42)
    train_images.extend(train_x)  # Add training images to the train list
    test_images.extend(test_x)  # Add testing images to the test list
    train_labels.extend(train_y)  # Add training labels to the train list
    test_labels.extend(test_y)  # Add testing labels to the test list

# Convert lists to numpy arrays
train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)


NameError: name 'class_names' is not defined

In [ ]:
def create_vit_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    # Patch embedding
    patch_size = 16
    num_patches = (input_shape[0] // patch_size) * (input_shape[1] // patch_size)
    patches = layers.Conv2D(64, patch_size, patch_size)(inputs)
    patches = layers.Reshape((num_patches, -1))(patches)

    # Positional embedding
    positional_embedding = layers.Embedding(input_dim=num_patches, output_dim=64)(tf.range(num_patches))
    x = patches + positional_embedding

    # Transformer blocks
    for _ in range(4):
        attention_output = layers.MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
        x = layers.Add()([x, attention_output])
        x = layers.LayerNormalization()(x)
        ff_output = layers.Dense(128, activation='relu')(x)
        ff_output = layers.Dense(64)(ff_output)
        x = layers.Add()([x, ff_output])
        x = layers.LayerNormalization()(x)

    # Classification head
    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    return models.Model(inputs, outputs)


In [ ]:
# Create the ViT model
input_shape = (224, 224, 1)  # Input shape for grayscale images
num_classes = len(class_names)  # Number of classes (tumor types + no tumor)
model = create_vit_model(input_shape, num_classes)  # Build the model


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # Compile with Adam optimizer

# Step 3: Train the model
history = model.fit(train_images, train_labels, validation_data=(test_images, test_labels), epochs=20, batch_size=32)  # Train the model

# Step 4: Evaluate the model
predictions = model.predict(test_images)  # Predict on the test set
predicted_labels = np.argmax(predictions, axis=1)  # Convert probabilities to class labels


In [ ]:
# Classification report
print("Classification Report:")
print(classification_report(test_labels, predicted_labels, target_names=class_names))  # Print detailed metrics

# Confusion matrix
cm = confusion_matrix(test_labels, predicted_labels)  # Compute confusion matrix
plt.figure(figsize=(12, 10))  # Set figure size
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)  # Plot heatmap
plt.xlabel('Predicted Labels')  # Label for x-axis
plt.ylabel('True Labels')  # Label for y-axis
plt.title('Confusion Matrix')  # Title of the plot
plt.show()  # Display the plot



In [ ]:
# Step 5: Plot training and validation metrics
def plot_metrics(history):
    plt.figure(figsize=(12, 5))

    # Accuracy
    if 'accuracy' in history.history and 'val_accuracy' in history.history:
        plt.subplot(1, 2, 1)
        plt.plot(history.history['accuracy'], label='Training Accuracy')
        plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
        plt.title('Accuracy Over Epochs')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()

    # Loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss Over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()